<a href="https://colab.research.google.com/github/JohnnyKyr/CompIntel-Projects/blob/main/GAs2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
from math import log2
import random
import numpy as np
import math
from scipy import sparse

In [28]:
class Parser:

    def __init__(self):
        self.dataMatrix = None
        self.labels = None
        self.size = None

    def getLexicalSize(self,file):
        lexiconS = []
        with open(file, "r") as fp:
            for line in fp:
                lexiconS.append(line.split()[0])
        return  len(lexiconS)
    def getLabels(self,data):
        self.labels = []
        with open(data, "r") as fp:
            for line in fp:
                self.labels.append([int(i) for i in line.split()])

    def getData(self,data,lexicon):
        bagof = []
        bagofcsr =[]
        with open(data,"r") as fp:
            for line in fp:
                vertex = [0 for i in range(self.getLexicalSize(lexicon))]
                array = list(filter(lambda x : x[0] != '<' , line.split()))
                for ele in array:
                    vertex[int(ele)] +=1
                self.size = len(vertex)
                bagof.append(vertex)
        return bagof


In [29]:

datalex = Parser()


bagofwords = datalex.getData("/drive/MyDrive/ML/train-data.dat","/drive/MyDrive/ML/vocabs.txt")



In [ ]:
from os import WCOREDUMP
class Individual(object):
    def __init__(self, chromosome):
        self.chromosome = chromosome
        

    @classmethod
    def create_genome(self):

        genome = np.random.randint(0, 1, size=8520)
        rand_ones = random.randint(0, 8519)
        for _ in range(rand_ones):
            genome[random.randint(0, 8519)] = 1
        return genome

    def calculate_fitness(self):
        return 0


def genome_rejection(genome: Individual):
    # replace the non valid genome
    # TODO recursion
    while np.count_nonzero(genome) < 1000:
        genome = Individual.create_genome()
    return genome


def repair_genome(genome: Individual):
    nnz = np.count_nonzero(genome)
    while nnz < 1000:
        difference = 1000 - nnz
        for _ in range(difference):
            genome[random.randint(0, 8519)] = 1
        nnz = np.count_nonzero(genome)
    return genome

def initialize_population(no_of_chromosomes):
    population = []
    for _ in range(no_of_chromosomes):
        genome = Individual.create_genome()
        # genome = genome_rejection(genome)
        genome = repair_genome(genome)
        population.append(Individual(genome))

    return population


def fitnessFunc(population:Individual,tfidf):
  #Calculating the fittest with the best sum of avg in tf_idf
  #Sort population by nonzero
  population.sort(key = lambda x:np.count_nonzero(x.chromosome))
  significance = [0 for _ in range(len(population))]

  for i,individual in enumerate(population):
    for value in np.nonzero(individual.chromosome)[0]:
      significance[i] += tfidf[value]
      
  return significance


def tf_idf(bagofwords,verbose=0):
  count=0
  N=len(bagofwords)
  n = len(bagofwords[0])
  tfbag = []
  
  textDCount = np.count_nonzero(bagofwords,axis=0)  #word count of the text body
  worddcount = np.count_nonzero(bagofwords,axis=1) #word count for each text
  
  #----------------IDF----------------------
  _idf = [0 for _ in range(len(textDCount))]
  tf_idf= []
  for k,value in enumerate(textDCount):
    _idf[k] = np.log10(N/value)
  _idf = np.array(_idf)
  #---------------TF-----------------------
  tfDict = [0 for _ in range(N)]
  for i in range(N):
    count+=1
    tfDict[i] = np.array([(text/worddcount[i]) for k,text in enumerate(bagofwords[i])])
    if verbose==1:
      print("======================================")
      print("Finished",count,"st","Calculation")
  
  tfDict = np.array(tfDict)
  
  
  for ele in tfDict:
    tf_idf.append(ele*_idf)
  
  avg = np.mean(tf_idf, axis = 0)
  return avg
 


  
   

tf__idf = tf_idf(bagofwords)
population = initialize_population(10)
fitness = fitnessFunc(population,tf__idf)